# Study of a more simple trajectory
We will study a collection of 3 trajectories in order to compare the visualisation of each library

In [1]:
"""
Function to compute the execution time.
"""
from datetime import timedelta
import time
def get_time(f, kwargs={}):
    times = []
    result = None
    for i in range(1):
        print("\nmeasure",i+1)
        t = time.time()
        result = f(**kwargs)
        res = time.time()-t
        times.append(res)
        print( timedelta(seconds=res))
        if res > 3600*2:
            print("trop de temps")
            return result
        
    print("\nall times:", times)
    a = min(times)
    b = max(times)
    print("times' outliers:",a,"&",b)
    t = abs(sum(times)-a-b)/1
    print("benchmark time",str(t)+"s","or", timedelta(seconds=t))
    return result

In [2]:
import numpy as np
import pandas as pd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import movingpandas as mpd
#from holoviews import opts, dim
# import hvplot
import warnings
warnings.simplefilter("ignore")

In [3]:
SIZE = "small"

# Ships Trajectories

In [4]:
PATH='../../datasets/ships_'+SIZE+'.csv'
# PATH = "result_ptrail.csv"
df = pd.read_csv(PATH)
traj = mpd.TrajectoryCollection(data=df, traj_id_col='id', t='Timestamp', x='Longitude', y='Latitude')

In [ ]:
traj.to_point_gdf()

### Outlier Detection IQR

In [ ]:
traj.add_speed(True)

In [ ]:
cleaned_traj = get_time(mpd.IqrCleaner(traj).clean,{"columns":{'speed': 3}})

In [ ]:
print("nombre precedent de points:", len(traj.to_point_gdf()))
print("nombre de points retirés:", len(traj.to_point_gdf())-len(cleaned_traj.to_point_gdf()), "or", len(df) - len(cleaned_traj.to_point_gdf()))
print("nombre actuel de points:", len(cleaned_traj.to_point_gdf()), cleaned_traj.to_point_gdf().shape)

### Outlier Detection speed Threshold

In [5]:
cleaned_traj = get_time(mpd.OutlierCleaner(traj).clean,{"v_max":73, "units":("m", "s")})


measure 1
211.2284290790558 and 137.58285641670227
copy time 1.6927719116210938e-05
0:03:32.256513

all times: [212.25651264190674]
times' outliers: 212.25651264190674 & 212.25651264190674
benchmark time 212.25651264190674s or 0:03:32.256513


In [6]:
print("nombre precedent de points:", len(traj.to_point_gdf()))
print("nombre de points retirés:", len(traj.to_point_gdf())-len(cleaned_traj.to_point_gdf()), "or", len(df) - len(cleaned_traj.to_point_gdf()))
print("nombre actuel de points:", len(cleaned_traj.to_point_gdf()), cleaned_traj.to_point_gdf().shape)

nombre precedent de points: 1208642
nombre de points retirés: 249 or 37956
nombre actuel de points: 1208393 (1208393, 7)


# cars trajectories

In [ ]:
PATH='../datasets/cars_'+SIZE+'.csv'
df = pd.read_csv(PATH)
traj = mpd.TrajectoryCollection(data=df, traj_id_col='id', t='Timestamp', x='Longitude', y='Latitude')

In [ ]:
traj.add_speed(True)
cleaned_traj = mpd.OutlierCleaner(traj).clean({'speed': 1.5})

In [ ]:
print("nombre precedent de points:", len(traj.to_point_gdf()))
print("nombre de points retirés:", len(traj.to_point_gdf())-len(cleaned_traj.to_point_gdf()))
print("nombre actuel de points:", len(cleaned_traj.to_point_gdf()), cleaned_traj.to_point_gdf().shape)

# Accuracy

In [ ]:
PATH="../Dataset/ground_truth_medium.csv"
df = pd.read_csv(PATH)
df